## This is the script to generate Figure 1b, 2b, 1c, 2c and Figure S2

In [2]:
library(ggplot2)
library(dplyr)
library(ggstatsplot)
library(grid)
library(ggthemes)
library(RColorBrewer)
library(ggpubr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“replacing previous import ‘dplyr::collapse’ by ‘glue::collapse’ when loading ‘statsExpressions’”
You can cite this package as:
     Patil, I. (2021). Visualizations with statistical details: The 'ggstatsplot' approach.
     Journal of Open Source Software, 6(61), 3167, doi:10.21105/joss.03167



In [235]:
#ggplot publication ready theme
theme_Publication <- function(base_size = pt_sz, base_family=family_sz) {
      library(grid)
      library(ggthemes)
      (theme_foundation(base_size=base_size, base_family=base_family)
       + theme(plot.title = element_text(face = "bold",
                                         size = rel(1.2), hjust = 0.5),
               text = element_text(),
               panel.background = element_rect(colour = NA),
               plot.background = element_rect(colour = NA),
               panel.border = element_rect(colour = NA),
               axis.title = element_text(size = rel(1)),
               axis.title.y = element_text(angle=90,vjust =2),
               axis.title.x = element_text(vjust = -0.2),
               axis.text = element_text(), 
               axis.line = element_line(colour="black"),
               axis.ticks = element_line(),
               panel.grid.major = element_line(colour="#f0f0f0"),
               panel.grid.minor = element_blank(),
               legend.key = element_rect(colour = NA),
               legend.position = "bottom",
               legend.direction = "horizontal",
               legend.key.size= unit(0.2, "cm"),
               legend.margin = unit(0, "cm"),
               legend.title = element_text(face="italic"),
               plot.margin=unit(c(10,5,5,5),"mm"),
               strip.background=element_rect(colour="#f0f0f0",fill="#f0f0f0"),
               strip.text = element_text(face="bold")
          ))
      
}

scale_fill_Publication <- function(...){
      library(scales)
      discrete_scale("fill","Publication",manual_pal(values = c("#386cb0","#fdb462","#7fc97f","#ef3b2c","#662506","#a6cee3","#fb9a99","#984ea3","#ffff33")), ...)

}

scale_colour_Publication <- function(...){
      library(scales)
      discrete_scale("colour","Publication",manual_pal(values = c("#386cb0","#fdb462","#7fc97f","#ef3b2c","#662506","#a6cee3","#fb9a99","#984ea3","#ffff33")), ...)

}

In [236]:
dir <- "/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2021/velocityRNA/results/combined_analysis/dge_data_collection"
setwd(dir)

In [238]:
#read the data of mean delta fold change by comparison
file <- "fc_stat_stat/cat.fc_stat.txt"
rt <- read.csv(file, header=F, sep="\t")
colnames(rt) <- c("study", "delta_fc", "pval", "group", "reads", "sampling", "tool", "cpm")
#SRP045666 is not disease condition
#command to find the number of genes used to calculate the mean delta fold change
#l /Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2021/velocityRNA/results/combined_analysis/dge_data_collection/stat_collected/abnormal_conditions.all_reads.DESeq2.logCPM5.fc_data|cut -f1|sort|uniq -c|sort -k1,1n|l
lt_30 <- c("SRP032165", "SRP047194.d11", "SRP043221", "SRP045666")
lt_50 <- c(lt_30, c("SRP047194.d31", "SRP063059"))
#will used in Median delta fold change calculation
#mean delta fold change will only be calculated on comparisons with >= 50 genes
exclude_list <- lt_50

filterd_passsed <- unique (grep(paste(exclude_list,collapse="|"), rt$study, ignore.case=T, invert=T))
rt <- rt[filterd_passsed,]

#print different kinds of experiments in comparisons
for(i in 4:8){
    print(unique(rt[,i]))
    print("\n")
}

[1] "Abnormal_conditions"    "GTEx_tissues"           "GTEx_tissues_5_samples"
[4] "Non-human_tissues"      "SRA_tissues"           
[1] "\n"
[1] "3prime_reads"     "5prime_reads"     "All_mapped_reads" "Boundary_reads"  
[1] "\n"
[1] "All_reads"  "Subsampled"
[1] "\n"
[1] "DESeq2" "EdgeR" 
[1] "\n"
[1] "LogCPM5" "LogCPM0"
[1] "\n"


In [239]:
#Change the group names to reomve underscores
change_group_names <- function(dat){
    dat$group[dat$group=="Abnormal_conditions" | dat$group=="abnormal_conditions"] <- "Diseases"
    dat$group[dat$group=="GTEx_tissues" | dat$group=="gtex_tissue"] <- "GTEx tissues"
    dat$group[dat$group=="Non-human_tissues" | dat$group=="non-human_species"] <- "Non-human\ntissues"
    dat$group[dat$group=="SRA_tissues" | dat$group=="sra_tissues"] <- "SRA tissues"
    dat$group[dat$group=="GTEx_tissues_5_samples" | dat$group=="gtex_tissue_5_samples"] <- "GTEx tissues\n(5 samples)"

    dat$group <- factor(dat$group, levels=c("GTEx tissues", "GTEx tissues\n(5 samples)", "SRA tissues", "Non-human\ntissues", "Diseases"))
    dat
}

In [240]:
#output folder
vis_dir <- "/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2021/velocityRNA/results/combined_analysis/visualize4paper/Figures/"

In [241]:
#load shared colors or values
source("/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2021/velocityRNA/results/combined_analysis/visualize4paper/shared/shared.R")
#plot Figure 1b and 2b
plot_hist <- function(dat, col){
    gghistogram(dat, x="delta_fc", 
            add = "mean", rug = FALSE,
            col = col,
            palette = c("npg"),
            xlab="Mean delta fold change",
            ylab="Density",
            size=0.65
  ) + geom_vline(xintercept=0, linetype="dashed", color = "#060606", size=0.75) +
    theme_Publication(base_family=family_sz, 
                      base_size = pt_sz
                     ) +
      
      theme(legend.position = "none",
           panel.grid.major = element_blank(),
            axis.line = element_line(size=0.4),
            axis.title = element_text(face = NULL,size = rel(1))
           
           ) +xlim(c(-0.75, 0.75))
}

dat <- rt %>% filter(rt$reads == "All_mapped_reads" & rt$sampling == "Subsampled" & rt$tool == "DESeq2" & rt$cpm == "LogCPM5")
dat_all_reads <- change_group_names(dat)

pdf(paste0(vis_dir, "Figure1b.pdf"), width=2.13, height=2.13, pointsize=pt_sz)

plot_hist(dat_all_reads[dat_all_reads$group=="GTEx tissues",], col=col_sz[1])
dev.off()


pdf(paste0(vis_dir, "Figure2b.pdf"), width=2.13, height=2.13, pointsize=pt_sz)
plot_hist(dat_all_reads[dat_all_reads$group=="Diseases",], col=col_sz[4])

dev.off()
#brewer.pal(n = 8, name = "Dark2")


Warning message:
“Using `bins = 30` by default. Pick better value with the argument `bins`.”
Warning message:
“geom_vline(): Ignoring `mapping` because `xintercept` was provided.”
Warning message:
“geom_vline(): Ignoring `data` because `xintercept` was provided.”
Warning message:
“`legend.margin` must be specified using `margin()`. For the old behavior use legend.spacing”


pdf 
  2

Warning message:
“Using `bins = 30` by default. Pick better value with the argument `bins`.”
Warning message:
“geom_vline(): Ignoring `mapping` because `xintercept` was provided.”
Warning message:
“geom_vline(): Ignoring `data` because `xintercept` was provided.”
Warning message:
“`legend.margin` must be specified using `margin()`. For the old behavior use legend.spacing”


pdf 
  2

In [242]:
#function to generate box plot and violin plot
plot_ggbetweenstats <- function(dat, cols){
    plt <-ggbetweenstats(
      data = dat,
      x = group,
      y = delta_fc,
      pairwise.comparisons=FALSE,
      bf.message = FALSE,
      results.subtitle = FALSE,
      xlab = "",
      ylab = "Mean delta fold change",
      centrality.plotting = FALSE,
      point.args = list(position = ggplot2::position_jitterdodge(dodge.width = 0.6),size=1.35,alpha=0.8, stroke=0)
    )
    plt + geom_hline(yintercept=0, linetype="dashed", color = "#060606", size=0.75) + 
      theme_Publication() +
      scale_color_manual(values=cols) +
      theme(legend.position = "none",
           panel.grid.major = element_blank()
           )
    
}

#options(repr.plot.width=4, repr.plot.height=2)
pdf(paste0(vis_dir, "/Figure1c.pdf"), width=3.5, height=2.2)
par(mar=c(0.2,0.1,0,0.5))
dat <- rt %>% filter(rt$group != "GTEx_tissues_5_samples" & rt$group != "Abnormal_conditions" & rt$reads == "All_mapped_reads" & rt$sampling == "Subsampled" & rt$tool == "DESeq2" & rt$cpm == "LogCPM5")
dat_all_reads <- change_group_names(dat)
plot_ggbetweenstats(dat_all_reads,col_sz[c(1,2,3)])
dev.off()

Warning message:
“`legend.margin` must be specified using `margin()`. For the old behavior use legend.spacing”
Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.



pdf 
  2

In [278]:
dat <- rt %>% filter(rt$group != "GTEx_tissues_5_samples" & rt$reads == "All_mapped_reads" & rt$sampling == "Subsampled" & rt$tool == "DESeq2" & rt$cpm == "LogCPM5")
dat_all_reads <- change_group_names(dat)

options(repr.plot.width=4, repr.plot.height=2)
pdf(paste0(vis_dir, "/Figure2c.pdf"), width=4.8, height=2.2)
par(mar=c(0.2,0.1,0,0.5))

plot_ggbetweenstats(dat_all_reads,col_sz[1:4])
dev.off()



Warning message:
“`legend.margin` must be specified using `margin()`. For the old behavior use legend.spacing”
Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.



pdf 
  2

In [277]:
#test if mean delta fold change in each of tissue group is significantly greater than zero, or disease is significantly less than zero
for(group in unique(dat$group)){
    print(paste("test", group))
    one_group_dat <- dat$delta_fc[dat$group==group]
    alt <- "less"
    if(median(one_group_dat) > 0){
        alt <- "greater"
    }

    if(shapiro.test(one_group_dat)$p.value > 0.05){
        print("one sample t-test used")
        print(t.test(one_group_dat, mu=0, alternative = alt))
    }else{
        print("one sample wilcox test used")
        print(wilcox.test(one_group_dat, mu = 0, alternative = alt))
    }
    print("");print("")
}


#

[1] "test Abnormal_conditions"
[1] "one sample t-test used"

	One Sample t-test

data:  one_group_dat
t = -7.2643, df = 42, p-value = 3.051e-09
alternative hypothesis: true mean is less than 0
95 percent confidence interval:
       -Inf -0.1671238
sample estimates:
 mean of x 
-0.2174778 

[1] ""
[1] ""
[1] "test GTEx_tissues"
[1] "one sample wilcox test used"

	Wilcoxon signed rank test with continuity correction

data:  one_group_dat
V = 153929, p-value < 2.2e-16
alternative hypothesis: true location is greater than 0

[1] ""
[1] ""
[1] "test Non-human_tissues"
[1] "one sample wilcox test used"

	Wilcoxon signed rank exact test

data:  one_group_dat
V = 421, p-value = 1.344e-05
alternative hypothesis: true location is greater than 0

[1] ""
[1] ""
[1] "test SRA_tissues"
[1] "one sample wilcox test used"

	Wilcoxon signed rank test with continuity correction

data:  one_group_dat
V = 11297, p-value = 1.049e-14
alternative hypothesis: true location is greater than 0

[1] ""
[1] ""


In [244]:
#function to plot boxplot in Figure S2
plot_boxplot <-function(dat, fill_group, legend_title, cols=col_sz, ylab="Mean delta fold change", legend.position="top"){
    if(fill_group=="group"){
        legend.position="none"
    }
    p <- ggplot(data = dat, aes(x=group, y=delta_fc)) + 
     geom_boxplot(aes_string(fill=fill_group), lwd=0.26, outlier.size = 0.4)


    p + geom_hline(yintercept=0, linetype="dashed", color = "#060606", size=0.6) + 
          theme_Publication() +
          scale_color_manual(values=cols) +
          scale_fill_manual(values=cols) +
          theme(panel.grid.major = element_blank(), 
                legend.key.size = unit(0.3, 'cm'),
                legend.position=legend.position
               ) +
          guides(fill=guide_legend(title=legend_title)) +
          labs(y=ylab, x="")
}


pdf(paste0(vis_dir, "/FigureS2d.pdf"), width=4, height=2.5)
#options(repr.plot.width=7, repr.plot.height=5)
dat <- rt %>% filter(rt$group != "GTEx_tissues_5_samples" & rt$group != "Non-human_tissues" & rt$sampling == "Subsampled" & rt$tool == "DESeq2" & rt$cpm == "LogCPM5")
dat <- change_group_names(dat)
dat$reads <- gsub("_", " ", dat$reads)
plot_boxplot(dat, "reads", "Reads")
dev.off()

Warning message:
“`legend.margin` must be specified using `margin()`. For the old behavior use legend.spacing”


pdf 
  2

In [245]:
pdf(paste0(vis_dir, "/FigureS2c.pdf"), width=4, height=2.5)
dat <- rt %>% filter(rt$group != "GTEx_tissues_5_samples" & rt$reads=="All_mapped_reads"  & rt$tool == "DESeq2"  & rt$cpm == "LogCPM5")
dat <- change_group_names(dat)
dat$sampling <- gsub("_", " ", dat$sampling)
plot_boxplot(dat, "sampling", "Read sampling")


dev.off()


Warning message:
“`legend.margin` must be specified using `margin()`. For the old behavior use legend.spacing”


pdf 
  2

In [246]:
pdf(paste0(vis_dir, "/FigureS2a.pdf"), width=4, height=2.5)
dat <- rt %>% filter(rt$group != "GTEx_tissues_5_samples" & rt$reads=="All_mapped_reads"  & rt$cpm == "LogCPM5")
dat <- change_group_names(dat)
plot_boxplot(dat, "tool", "Tool")

dev.off()

Warning message:
“`legend.margin` must be specified using `margin()`. For the old behavior use legend.spacing”


pdf 
  2

In [247]:
pdf(paste0(vis_dir, "/FigureS2b.pdf"), width=4, height=2.5)
dat <- rt %>% filter(rt$group != "GTEx_tissues_5_samples" & rt$reads=="All_mapped_reads" & rt$tool == "DESeq2")
dat <- change_group_names(dat)
p <- plot_boxplot(dat, "cpm", "CPM filtering")
p + scale_fill_manual(name = "CPM filtering", labels = c("log2CPM > 0", "log2CPM > 5"),values=col_sz)

dev.off()

Warning message:
“`legend.margin` must be specified using `margin()`. For the old behavior use legend.spacing”
Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.



pdf 
  2

In [248]:
pdf(paste0(vis_dir, "/FigureS2e.pdf"), width=4, height=2.5)
dat <- rt %>% filter(rt$reads == "All_mapped_reads" & rt$sampling == "Subsampled" & rt$tool == "DESeq2" & rt$cpm == "LogCPM5")
dat <- change_group_names(dat)
plot_boxplot(dat, "group", "Group", col_sz[c(1,5,2,3,4)])

dev.off()

Warning message:
“`legend.margin` must be specified using `margin()`. For the old behavior use legend.spacing”


pdf 
  2

In [252]:
data_dir <- "/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2021/velocityRNA/results/combined_analysis/dge_data_collection/stat_collected"


In [253]:
#calculate the median (not mean) delta fold change for each comparison 
name <- "abnormal_conditions"
dat <- data.frame(group=NULL, delta_fc=NULL)
for(name in c("abnormal_conditions", "non-human_species", "sra_tissues", "gtex_tissue")){

    fc_data <- read.csv(paste0(data_dir, "/", name, ".all_reads.DESeq2.logCPM5.fc_data"), header=F, sep="\t")
    colnames(fc_data) <- c("study", "ens", "e_fc", "i_fc")
    if(name == "abnormal_conditions"){
        filterd_passsed <- unique (grep(paste(exclude_list,collapse="|"), fc_data$study, ignore.case=T, invert=T))
        fc_data <- fc_data[filterd_passsed, ]
    }
    delta_fc <- aggregate(abs(fc_data$e_fc) - abs(fc_data$i_fc), by=list(fc_data$study), median)[,2]
    dat <- rbind(dat, data.frame(group=name, delta_fc=delta_fc))
#plot(density(delta_fc), main=name)
}


In [254]:
#generate Figure S2f

dat <- change_group_names(dat)
pdf(paste0(vis_dir, "/FigureS2f.pdf"), width=4, height=2.5)
plot_boxplot(dat, "group", "Group", ylab="Median delta fold change")
dev.off()

Warning message:
“`legend.margin` must be specified using `margin()`. For the old behavior use legend.spacing”


pdf 
  2